## Preparation

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

## Site: levels.fyi

In [2]:
page = requests.get('https://www.levels.fyi/companies')
soup = BeautifulSoup(page.content, 'html.parser')
companies_container = soup.find_all(
    'h6', class_='MuiTypography-root MuiTypography-h6 css-1v6gvkr')
companies = [company.text for company in companies_container]
companies

['Google',
 'Amazon',
 'Apple',
 'Lyft',
 'Facebook',
 'Microsoft',
 'Uber',
 'Stripe',
 'Roblox',
 'Coinbase',
 'Databricks',
 'Snap',
 'Netflix',
 'LinkedIn',
 'Salesforce',
 'Hudson River Trading',
 'Jane Street',
 'Citadel',
 'Two Sigma',
 'JPMorgan Chase',
 'Capital One',
 'Oracle',
 'Bytedance',
 'Intel']

In [ ]:
def crawl_company(df: pd.DataFrame, company_id: str):
	page = requests.get(f'https://www.levels.fyi/companies/{company_id}')
	soup = BeautifulSoup(page.content, 'html.parser')
	size = soup.findAll(
		'h6', class_='MuiTypography-root MuiTypography-subtitle1 css-idrr7q')[1].text

	page = requests.get(f'https://www.levels.fyi/companies/{company_id}/salaries')
	soup = BeautifulSoup(page.content, 'html.parser')
	job_titles_container = soup.find_all('h6', class_='MuiTypography-root MuiTypography-h6 css-jv9qtm')
	job_titles = [job_title.text for job_title in job_titles_container]
	job_title_ids = [job_title.strip().lower().replace(' ', '-') for job_title in job_titles]

	for job_title_id in job_title_ids:
		driver.get(f'https://www.levels.fyi/company/{company_id}/salaries/{job_title_id}')

		print(f'--> crawling {company_id} {job_title_id}...')

		try:
			button = driver.find_element(
				by=By.CSS_SELECTOR, value="button.MuiButton-root.MuiButton-text.MuiButton-textPrimary.MuiButton-sizeMedium.MuiButton-textSizeMedium.MuiButtonBase-root.css-um5318")
			button.click()

			button = driver.find_element(
				by=By.CSS_SELECTOR, value="button.MuiButton-root.MuiButton-text.MuiButton-textPrimary.MuiButton-sizeMedium.MuiButton-textSizeMedium.MuiButtonBase-root.css-g9gvkf")
			button.click()
		except:
			pass

		try:
			tbody = driver.find_element(
				by=By.CSS_SELECTOR, value="tbody.MuiTableBody-root.css-6l3v2e"
			)
			rows = tbody.find_elements(
				by=By.TAG_NAME, value="tr"
			)
			for row in rows:
				try:
					cells = row.find_elements(
						by=By.TAG_NAME, value="td"
					)
					location = cells[0].find_element(
						by=By.TAG_NAME, value="span"
					).find_element(
						by=By.TAG_NAME, value="span"
					).text.split('|')[0]
					level = cells[1].find_element(
						by=By.TAG_NAME, value="p"
					).text
					field = cells[1].find_element(
						by=By.TAG_NAME, value="span"
					).text
					yoe_total = cells[2].find_element(
						by=By.TAG_NAME, value="p"
					).text
					yoe_at_company = cells[2].find_element(
						by=By.TAG_NAME, value="span"
					).text
					total_compensation = cells[3].find_element(
						by=By.TAG_NAME, value="p"
					).text

					new_record = {
						'company': company_id,
						'company_size': size,
						'job_title': job_title_id,
						'level': level,
						'domain': field,
						'yoe_total': yoe_total,
						'yoe_at_company': yoe_at_company,
						'total_compensation': total_compensation,
						'location': location
					}

					df.loc[len(df)] = new_record
				except:
					continue
		except:
			pass
		
	


df = pd.DataFrame(columns=[
	'company',
	'company_size',
	'job_title',
	'level',
	'domain',
	'yoe_total',
	'yoe_at_company',
	'total_compensation',
	'location'
])

driver = webdriver.Chrome()

for company in companies:
	crawl_company(df, company)

driver.close()

df.to_csv('data.csv', index=False)

df